In [2]:
# reloading for modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# ignore some unwanted warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
# import the needed modules
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import sys

In [231]:
def k_clique_communities(G, k=3):
    # NOTE: a couple of stages could be combined, 
    #       but were left as is for ease of understanding
    
    # 1: find all maximal cliques
    cliques = list(nx.find_cliques(G))

    # 2: create clique overlap matrix
    num_of_cliques = len(cliques)
    matrix = np.zeros((num_of_cliques,num_of_cliques))
    for i in range(0,num_of_cliques):
        for j in range(0,num_of_cliques):
            for v in cliques[i]:
                if v in cliques[j]:
                    matrix[i,j] += 1
    
    # 3: threshold matrix with k
    for i in range(0,num_of_cliques):
        for j in range(0,num_of_cliques):
            if (matrix[i,j] < k-1) or (i == j and matrix[i,j] < k):
                matrix[i,j] = 0
            else:
                matrix[i,j] = 1
    
    # 4: combine connected components, generate communities
    communities = list()
    G_hier = nx.Graph()
    for i in range(0,num_of_cliques):
        for j in range(0,i+1):
            if matrix[i,j] == 1:
                G_hier.add_edge(i,j)
    for comm_idx in nx.connected_components(G_hier):
        comm = set()
        for v in comm_idx:
            comm |= set(cliques[v])
        communities.append(comm)
    
    return communities

In [233]:
from networkx.algorithms.community import k_clique_communities as nx_k_clique_communities

# Graph G
G = nx.read_edgelist("communities.txt")

# our algorithm implementation
user_comm = k_clique_communities(G,4)
# networkx implementation for validation
nx_comm = list(nx_k_clique_communities(G,4))

# verify the correctness
is_ok = True
for s in user_comm:
    if s not in nx_comm:
        is_ok = False
for s in nx_comm:
    if s not in user_comm:
        is_ok = False
if not is_ok:
    print("A difference exists.")
else:
    print("Implementation is correct.")

print("Communities:")
c_idx = 0
for c in user_comm:
    c_idx += 1
    print("\t#%d : %s" % (c_idx, c))

Implementation is correct.
Communities:
	#1 : {'275', '328', '78', '181', '306', '4', '218', '273', '152', '195'}
	#2 : {'75', '211', '82', '105', '199', '271', '223', '150', '246', '346', '126', '332', '236', '252', '324', '64', '60', '142', '94', '276', '196', '331', '302', '300', '338', '277', '59', '80', '197', '169', '340', '188', '29', '57', '303', '317', '146', '274', '139', '38', '265', '132', '221', '77', '308', '92', '257', '16', '178', '314', '229', '266', '172', '121', '170', '128', '168', '21', '298', '322', '10', '249', '290', '53', '158', '5', '123', '189', '330', '291', '165', '185', '63', '222', '130', '318', '224', '186', '26', '313', '203', '13', '67', '62', '295', '106', '9', '136', '200', '184', '248', '40', '161', '65', '212', '272', '171', '329', '250', '304', '119', '242', '118', '239', '156', '79', '163', '180', '281', '238', '133', '320', '141', '268', '48', '73', '22', '194', '31', '345', '88', '325', '96', '254', '108', '280', '347', '261', '251', '84', '25'